In [5]:
"""
FEniCS tutorial demo program: Linear elastic problem.
  -div(sigma(u)) = f
The model is used to simulate an elastic beam clamped at
its left end and deformed under its own weight.
"""

# Put E and nu into this solver...

from __future__ import print_function
from fenics import *
from dolfin import *
import numpy as np
from mshr import *
from IPython.display import HTML

# Scaled variables
W = 2; h = 0.5; H = 2;
nu_s = 0.3              # Poisson's ration
E = 6E6                 # Elastic modulus
rho_s = 1


# first lame constant
mu_s = Constant( E/(2.0*(1.0+nu_s)))
# second lame constant
lambda_s = E*nu_s/((1.0+nu_s)*(1.0-2.0*nu_s))

# nodes in structure have to match nodes of saved structure values. 
N_h = 16;
N_w = 64;
# Create mesh and define function space

#domain = Rectangle(Point(0.0, 0.0), Point(W, h))
#f_domain = Rectangle(Point(0.0, h), Point(W, H))
#s_domain = Rectangle(Point(0.0, 0.0), Point(W, h))

#domain.set_subdomain(1,domain)

#mesh = generate_mesh(domain, N)
mesh = RectangleMesh(Point(0.0, 0.0), Point(W,h), N_w, N_h)

tol = 1E-14

#mesh = RectangleMesh(Point(0, 0), Point(W, h), 10, 3)
V = VectorFunctionSpace(mesh, 'Lagrange', 1)

# Define boundary condition
tol = 1E-14

def clamped_boundary_left(x, on_boundary):
    return on_boundary and x[0] < tol

def clamped_boundary_right(x, on_boundary):
    return on_boundary and x[0] > W - tol

def clamped_boundary_bottom(x, on_boundary):
    return on_boundary and x[1] < tol

bc_left = DirichletBC(V, Constant((0, 0)), clamped_boundary_left)
bc_right = DirichletBC(V, Constant((0, 0)), clamped_boundary_right)
bc_bottom = DirichletBC(V, Constant((0, 0)), clamped_boundary_bottom)
bcs = [bc_left, bc_right, bc_bottom]
#bcs = [bc_left, bc_right]

# Mark boundaries
facets = FacetFunction("size_t", mesh)
facets.set_all(0)

fsi = CompiledSubDomain('near(x[1], h1)  && on_boundary', h1 = h)
fsi.mark(facets, 3)

cells_s = CellFunction('size_t', mesh)


dA = Measure('ds', domain = mesh, subdomain_data = facets)
dV = Measure('dx', domain = mesh, subdomain_data = cells_s)


# Define functions
u = Function(V)             # Displacement from previous iteration
v = TestFunction(V)         # Test function
du = TrialFunction(V)       # Incremental displacement
B = Constant((0.0, 0.0))    # Structure body force.
T = Constant((0.0, -1000000))    # Traction force on the boundary


u0_s = Function(V)
u00_s = Function(V)

Dim = mesh.topology().dim()
I = Identity(Dim)
d_geo = u.geometric_dimension()  # space dimension

# load sigma from fsi problem, project onto structure.
# loaded sigma that has already been projected...

#nodal_S_sigmaFSI = np.loadtxt('nodal_S_sigmaFSI', dtype = float)

nodal_S_sigma = np.loadtxt('nodal_S_sigma', dtype = float)

#S_sigmaFSI_coords = np.loadtxt('S_sigmaFSI_coords', dtype = float)

T_space = TensorFunctionSpace(mesh, 'P', 1)
#dofs_s_T = T_space.tabulate_dof_coordinates().reshape((T_space.dim(),-1))


#i_s_T = np.where((dofs_s_T[:,1] == h))[0]

#coords_s = dofs_s_T[i_s_T]
sigma_FSI = Function(T_space)
sigma_FSI.vector()[:] = nodal_S_sigma

# should check this...
#sigma_FSI_1 = project(sigma_FSI, T_space, solver_type = "mumps",\
#    form_compiler_parameters = {"cpp_optimize" : True, "representation" : "quadrature", "quadrature_degree" : 2} )

#dofs_s_V = V.tabulate_dof_coordinates().reshape((V.dim(),-1))
#i_s_V_L = np.where((dofs_s_V[:,0] == 0))[0]
#i_s_V_R = np.where((dofs_s_V[:,0] == W))[0]
#i_s_V_B = np.where((dofs_s_V[:,1] == 0))[0]

# NeoHookean solver:

# Kinematics
#F = I + grad(u) # deformation gradient
#C = F.T*F       # Right Cauch-Green tensor

# Invariants of deformation tensor
#Ic = tr(C)
#J = det(F)

# Stored strain energy density (compressible neo-Hookean model)
#psi = (mu_s/2)*(Ic - 3) - mu_s*ln(J) + (lambda_s/2)*(ln(J))**2

# Calculate T...
n = FacetNormal(mesh)
#T = dot(sigma_FSI, n)

#F = I + grad(u)
#T = J*inv(F)*sigma_FSI*n
#T = dot(T_1, n)

#T = J*sigma_FSI*inv(F).T

#T = Constant((-1, 0))      # Traction sigma.n
#T = Expression(('0','x[0]*(x[0]-W)'), W = W, degree = 1)

# Total potential energy
#Pi = psi*dx-dot(T, u)*dA(3) - dot(B, u)*dx
# First directional derivative of Pi about d in the direction of v

#Form_s = derivative(Pi, u, v)

# using indices like Abali's code:
i,j, k, l, m = indices(5)
delta = Identity(Dim)
f = Constant((0,0))
dt = 0.01

F_s = as_tensor( u[k].dx(i) + delta[k, i], (k, i) )
J_s = det(F_s)

C_s = as_tensor( F_s[k, i]*F_s[k, j], (i, j) )
E_s = as_tensor(1./2.*(C_s[i, j] - delta[i, j]), (i, j) )
S_s = as_tensor( lambda_s*E_s[k, k]*delta[i, j] + 2.*mu_s*E_s[i, j], (i, j))
#S_s = as_tensor( lambda_s*E[k, k]*delta[i, j] + 2.*mu_s*E[i, j], (i, j))
P_s = as_tensor( F_s[i, j]*S_s[k, j], (k, i) )

t_hat = as_tensor( J_s*inv(F_s)[k, j]*sigma_FSI[j, i]*n[k] , (i, ) )
#t_hat = T
#t_hat = Expression(('0','C*x[0]*(x[0]-W)'), W = W, degree = 1, C = 10000)


Form_s = ( rho_s*(u-2.*u0_s+u00_s)[i]/(dt*dt)*v[i] + P_s[k, i]*v[i].dx(k) - rho_s*f[i]*v[i] )*dV - \
         t_hat[i]*v[i]*dA(3)

#Form_s = derivative(P_s, u, v)
# Jacobian of the directional derivative Fd
Gain_s = derivative(Form_s, u, du)

begin("Computing structure displacement")

parameters["form_compiler"]["cpp_optimize"] = True
ffc_options = {"optimize": True}

solve(Form_s == 0, u, bcs, J = Gain_s, \
    solver_parameters  = {"newton_solver":{"linear_solver" : "mumps", "relative_tolerance" : 1e-3} }, \
    form_compiler_parameters = {"cpp_optimize" : True, "representation" : "quadrature", "quadrature_degree" : 2} )
#end()


# Plot solution
#plot(u, title='Displacement', mode='displacement')

# Plot stress
#s = sigma(d) - (1./3)*tr(sigma(u))*Identity(d_geo)  # deviatoric stress
#von_Mises = sqrt(3./2*inner(s, s))
#V1 = FunctionSpace(mesh_s, 'P', 1)#
#von_Mises = project(von_Mises, V1)
#plot(von_Mises, title='Stress intensity')

V1 = FunctionSpace(mesh, 'P', 1)
# Compute magnitude of displacement
#u_magnitude = sqrt(dot(u, u))
#u_magnitude = project(u_magnitude, V1)
#plot(u, 'Displacement magnitude')
#print('min/max u:',
#      u_magnitude.vector().array().min(),#
#      u_magnitude.vector().array().max())

# Save solution to file in VTK format
File('elasticity_neo/displacement.pvd') << u
#File('elasticity_new/von_mises.pvd') << von_Mises
#File('elasticity_new/magnitude.pvd') << u_magnitude

print(u, 'max and min:', 
     u.vector().array().min(),
     u.vector().array().max())
# Hold plot
interactive()
HTML(X3DOM().html(u))


f_212 max and min: -4.90053484004e-08 3.54821472118e-09


In [52]:
print(nodal_S_sigmaFSI)
#print(S_sigmaFSI_coords)
#print(coords_s)
#coords_s.max()
#coords_s.min()

[ 0.12045902  0.00136999  0.00458471  0.23421559  0.50924421 -0.02572751
 -0.04272979 -0.19455963  0.28851847 -0.08291664 -0.05956448  0.08280954
  0.31984076 -0.10505256 -0.11787162 -0.0116467   0.49925309 -0.02408089
 -0.04148696 -0.13628477  0.24875095 -0.15304616 -0.12661999  0.11064771
  0.30865924 -0.14890467 -0.16379385 -0.01206427  0.46777085 -0.07579459
 -0.09219725 -0.11872476  0.22043725 -0.20026421 -0.17425806  0.11797806
  0.2719048  -0.19591972 -0.21064702 -0.00449216  0.43101738 -0.11952028
 -0.13592961 -0.11319711  0.1739726  -0.25208829 -0.22656618  0.12802339
  0.24681701 -0.20577623 -0.2181091  -0.02129181  0.39421924 -0.0851542
 -0.10211558 -0.14500839  0.00302351 -0.52800895 -0.51504088  0.26997279
  0.25060813 -0.133381   -0.13955828 -0.08547845  0.29358033 -0.25036891
 -0.27091902 -0.07478014  0.08482293 -0.2916072  -0.26389251  0.10917007
  0.10802664 -0.3074426  -0.3228169   0.00146629  0.27082963 -0.21816912
 -0.23439938 -0.11502363  0.00784539 -0.33702018 -0.

In [10]:
ls


elasticity_neo/         elasticity_new/    elasticity_orig.py
elasticity_neo.py       elasticity_new.py
elasticity_neo_2.ipynb  elasticity_orig/


In [19]:
dofs_s_T.shape




(12732, 2)

In [20]:
nodal_S_sigmaFSI.shape

(4420,)